以下の設定になっていること  

ランタイム > ランタイムのタイプを変更  
ハードウェアアクセラレータ：GPU  
ランタイムの仕様：ハイメモリ  

編集 > ノートブックの設定  
ハードウェアアクセラレータ：GPU  
ランタイムの仕様：ハイメモリ  

In [1]:
# googleドライブをマウント
from google.colab import drive 
drive.mount('/content/drive')
# 表示されるリンクをクリックして、アクセスを許可して、最後に表示される文字列を以下の入力欄に入れる

Mounted at /content/drive


In [2]:
# 作業フォルダに移動
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


In [3]:
# importで使う必要があるので、インストールがランタイム切れるごとに必要
# インストール後にランタイムの再起動を行わないとT5Tokenizerが見つからない
# メニュー「ランタイム → ランタイムを再起動」で「Google Colab」を再起動

# ドライブに保存してるものでインストール
!pip install -e transformers

# Huggingface Datasetsのインストール
!pip install datasets==1.2.1

# Sentencepieceのインストール
!pip install sentencepiece==0.1.91

Obtaining file:///content/drive/My%20Drive/work/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 7.3 MB/s 
     |████████████████████████████████| 895 kB 75.9 MB/s 
  Running setup.py develop for transformers
     |████████████████████████████████| 159 kB 9.4 MB/s 
     |████████████████████████████████| 69 kB 10.3 MB/s 
     |████████████████████████████████| 243 kB 92.5 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.0
    Uninstalling tqdm-4.62.0:
      Successfully uninstalled tqdm-4.62.0
     |████████████████████████████████| 1.1 MB 8.3 MB/s 


In [1]:
# 作業フォルダに移動
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


CLM（Causal Language Modeling）: GPT、GPT-2

・model_name_or_path: モデルのチェックポイント（モデルを最初から学習しない場合）  
・model_type: モデルの種別（モデルを最初から学習する場合）  
・config_name: コンフィグ名（model_nameと同じでない場合）  
・tokenizer_name: トークナイザー名（model_nameと同じでない場合）  
・cache_dir: キャッシュフォルダ  
・use_fast_tokenizer: Fastトークナイザーを使用するかどうか  
・model_revision: 使用するモデルの特定のバージョン  
・use_auth_token: 「transformers-cli login」の実行時に生成されたトークンを使用するかどうか  

・dataset_name: データセット名  
・dataset_config_name: データセットのコンフィグ名  
・train_file: 学習データ（テキストファイル）  
・validation_file: 検証データ（テキストファイル）  
・overwrite_cache: キャッシュの上書き  
・validation_split_percentage: 学習データから使われる検証データの割合（検証データがない場合）  
・max_seq_length: トークン化後の最大合計入力シーケンス長  
・preprocessing_num_workers: 前処理に使用するプロセス数  
・block_size: トークン化後のオプションの入力シーケンス長  
・max_train_samples: 学習データの最大数  
・max_val_samples: 検証データの最大数  

    # GPT2のモデルファイルを指定
    --model_name_or_path=rinna/japanese-gpt2-medium \
    # 学習ファイル
    --train_file=train.txt \
    # 評価データファイル
    --validation_file=train.txt \
    # トレーニングを実施する
    --do_train \
    # 評価を実施する
    --do_eval \
    # 学習回数（エポック数）
    --num_train_epochs=30 \
    # チェックポイントの保存間隔
    --save_steps=5000 \
    # チェックポイントの保持数
    --save_total_limit=3 \
    # T5Tokenizer.model_max_length=1024をチャンクサイズとして使用するかblock_sizeで指定するかを設定する（設定しないとtokenizerの超大なサイズから1024になる）メモリに乗るように調整する必要がある（バッチサイズとの兼ね合い）
    --block_size=512 \
    # GPU1つあたりの学習バッチサイズ
    --per_device_train_batch_size=2 \
    # GPU1つあたりの評価バッチサイズ
    --per_device_eval_batch_size=2 \
    # モデルとチェックポイントの出力先
    --output_dir=output/ \
    # 出力先の上書きの許可
    --overwrite_output_dir=True \
    # T5Tokenizerで高速化ライブラリがあれば使用する
    --use_fast_tokenizer=False

### 学習する回数
エポック数 * (データ文字数 / ブロックサイズ):1つの学習データ / バッチサイズ  
    エポック数 = 1  
    データ文字数 = 22142  
    データの文字数だと数字が合わないので、tokenizeした結果の形態素数による？
    ブロックサイズ = 512  
    データ文字数 / ブロックサイズ = 43  
    バッチサイズ = 1  
    Total optimization steps = 29  


バッチサイズ = 2  
バッチサイズを2にすることで、トータルの実行回数が減っている  
Total optimization steps = 15  

In [2]:
%%time

# ファインチューニングの実行
!python ./transformers/examples/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=train_all.txt \
    --do_train \
    --num_train_epochs=500 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --block_size=256 \
    --per_device_train_batch_size=4 \
    --output_dir=output_all/ \
    --overwrite_output_dir=True \
    --use_fast_tokenizer=False

08/29/2021 12:01:46 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/29/2021 12:01:46 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=output_all/, overwrite_output_dir=True, do_train=True, do_eval=None, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=500.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Aug29_12-01-46_4f86858edcd0, logging_strategy=IntervalStrategy.STEPS, logging_first_step=False, logging_steps=500, save_strategy=IntervalStrategy.STEPS, save_steps=5000, save_total_limit=3, no_cuda=False, seed=42, fp16=False, fp16_opt_leve

In [ ]:
# from transformers import T5Tokenizer, AutoModelForCausalLM

# # トークナイザーとモデルの準備
# tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
# model = AutoModelForCausalLM.from_pretrained("output/")

In [3]:
# もちろんだが、Autoでも直指定でも同じ結果にはなっている
# https://huggingface.co/transformers/model_doc/gpt2.html#gpt2lmheadmodel
from transformers import T5Tokenizer, GPT2LMHeadModel

# GPU判定
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = GPT2LMHeadModel.from_pretrained("output_all/")
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [4]:
# 推論
# https://huggingface.co/blog/how-to-generate
# https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.generation_utils.GenerationMixin.generate
input = tokenizer.encode("左のジャブ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のジャブ チャット打ってきました続けデビュー戦ですが落ち着いた表情そして観客の拍手に応えるといったシーンもありました試合は十分といったところでしょう連打しますもしふらつきましたが大丈夫でしょうかでのパンチ集めて行きました。
左のジャブ やはりこの辺りは手数を増やしていく。大切なのはどういったところですか?
左のジャブ ピリピリとしたこの空気感とコンプレックスです本当にお互いがボクシング人生にかけているって言うならない選手は試合の序盤にダウンを奪われましたが第5ラウンドあたりからね自分の距離で戦えるようになる自分だけじゃないかもしれないですけどもあいつの強さで自分のペースに持って行こうとしているのがちょっと見えるかなと僕は思いまなるほど本来はもう着く距離を取り込んだと思う。
左のジャブ やはりこの辺りは手数を増やしていく。大切なのはどういったところですか?
左のジャブ プレッシャーをかけに来たのかと思われたのか右のストレートコンビネーションを見せてくれましたやり服大きなパンチが出ましたがこれは空を切りました。
左のジャブ やはりスピード感のある選手ですね。
左のジャブ コンビネーションを打つ時はしっかり踏み込んでパンツ売って行きますいらです左のボディお手数を増やしていったらという声も上がるほどなかなか折笠の捉えられないボクシング右のストレートコンビネーションで打ってきました。
左のジャブ やはりこの辺りは手数を増やしていく。大切なのはどういったところですか?
左のジャブ ピリピリとしたこの空気感とコンプレックスです本当にお互いがボクシング人生にかけているって言うならない選手は試合の序盤にダウンを奪われましたが第5ラウンドあたりからね自分の距離で戦えるようになる自分だけじゃないかもしれないですけどもあいつの強さで自分のペースに持って行こうとしているのがちょっと見えるかなと僕は思いまなるほど本来はもう着く距離を取り込んだと思う。
左のジャブ いきなり若干角度を変えながら右を出していきました今度はの左です。


In [5]:
input = tokenizer.encode("左のフック", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のフック みかとはよく寝つきを良くしていたんで吐いたりやりにくい距離感がピンボケするのではいよく言われる距離感が難しいというところなんですけども野上さんその辺りはいかがですかそうですねやはり常にオーソドックスと戦うことが多いのでそれを散歩に切り替えられたりっていうことはとてもやりにくい状況だと思いますもちろん鈴木奈々イモトのスイッチとは今回が初対戦になります第一ランドまもなくゴングという
左のフック 赤コーナーよりサカタケントがこれから入場をして来ようとしています。
左のフック 試合序盤からハグが積極的に前に行きますこのサウスポーの選手赤のグローブ昨年の2月にデビューしそこで判定負けを喫してしまった泡とみなされた。
左のフック しゃべる相手の動きを制します浜田ですただ前に前に入ろうとするのは堀これがデビュー戦です思い切って戦っているほりです前に出ようというところ歌詞でプレッシャーをかけて混ぜていく未来のジャムが入りました。
左のフック ボディに打ち込んでいく、そして、ショートレンジのパンチ。が入ってきてもパンチで応戦して、逆に押し返しているのがですかね。
左のフック スピード感のある選手ですね。
左のフック みかとはよく寝つきを良くしていたんで吐いたりやりにくい距離感がピンボケするのではいよく言われる距離感が難しいというところなんですけども野上さんその辺りはいかがですかそうですねやはり常にオーソドックスと戦うことが多いのでそれを散歩に切り替えられたりっていうことはとてもやりにくい状況だと思いますもちろん鈴木奈々イモトのスイッチとは今回が初対戦になります第一ランドまもなくゴングという
左のフック 赤コーナーよりサカタケントがこれから入場をして来ようとしています。
左のフック 試合序盤からハグが積極的に前に行きますこのサウスポーの選手赤のグローブ昨年の2月にデビューしそこで判定負けを喫してしまった泡とみなされた。
左のフック 少しふらついた!


In [6]:
input = tokenizer.encode("ボディ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ボディ 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。
ボディ 左でジャブを出していく青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。
ボディ 左のフック!左のボディ!右のストレート!も返していく!
ボディ 左のフック!左!ひかない!も引かない!右左!さあ今度はの逆襲!
ボディ 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。
ボディ 左でじわじわと詰めていきましたこの辺りの戦略というのはどうでしょうか距離に持って行きますがそうなるとかんだとしても入らせたくないジャムなども栗あってくるかもしれませんがここで右のストレートなたんだ左から右の番地を出して行きます右のボディを返して言ったのは長いです左のジャブが早いコナカですただこの間だが今回話していたのはその前に出るだけではなくていいとこ引いたような頭は
ボディ 左のフックを返してきました小西みかとも感じてるから第2ラウンド終了です。
ボディ 左のフックを見せました千葉です。
ボディ 左で距離を取りますヒヤリ。
ボディ 左のフックを返していった!


In [7]:
input = tokenizer.encode("右のストレート", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


右のストレート 試合序盤からハグが積極的に前に行きますこのサウスポーの選手赤のグローブ昨年の2月にデビューしそこで判定負けを喫してしまった泡とみなされた。
右のストレート 試合序盤からハグが積極的に前に行きますこのサウスポーの選手赤のグローブ昨年の2月にデビューしそこで判定負けを喫してしまった泡とみなされた。
右のストレート 試合序盤からハグが積極的に前に行きますこのサウスポーの選手赤のグローブ昨年の2月にデビューしそこで判定負けを喫してしまった泡とみなされた。
右のストレート 少し変化がありましたか。
右のストレート 試合序盤からハグが積極的に前に行きますこのサウスポーの選手赤のグローブ昨年の2月にデビューしそこで判定負けを喫してしまった泡とみなされた。
右のストレート 試合序盤からハグが積極的に前に行きますこのサウスポーの選手赤のグローブ昨年の2月にデビューしそこで判定負けを喫してしまった泡とみなされた。
右のストレート 試合序盤からハグが積極的に前に行きますこのサウスポーの選手赤のグローブ昨年の2月にデビューしそこで判定負けを喫してしまった泡とみなされた。
右のストレート 試合序盤からハグが積極的に前に行きますこのサウスポーの選手赤のグローブ昨年の2月にデビューしそこで判定負けを喫してしまった泡とみなされた。
右のストレート パリ最後まで体重が落ちませんしっかりて画像出して前に前にプレッシャーをかけて行きたいと言っていましたが残り10秒おまけに声出していいのなら明日の久保にもっと  ロッテに対する終了しましたがこちらが西桐生糸です22歳岐阜の横好きジム所属です今日が13戦目。
右のストレート 試合序盤からハグが積極的に前に行きますこのサウスポーの選手赤のグローブ昨年の2月にデビューしそこで判定負けを喫してしまった泡とみなされた。


In [8]:
input = tokenizer.encode("ガードの上から", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ガードの上から 早くもコンビネーション!
ガードの上から じわじわと圧力をかけていきますです。
ガードの上から じわじわと圧力をかけていくのは守り青葉。
ガードの上から いきなり大きなパンチを打ち込んでいきますです。
ガードの上から みかとはよく寝つきを良くしていたんで吐いたりやりにくい距離感がピンボケするのではいよく言われる距離感が難しいというところなんですけども野上さんその辺りはいかがですかそうですねやはり常にオーソドックスと戦うことが多いのでそれを散歩に切り替えられたりっていうことはとてもやりにくい状況だと思いますもちろん鈴木奈々イモトのスイッチとは今回が初対戦になります第一ランドまもなくゴングという
ガードの上から じわじわと五十嵐のプレッシャーをかけていきます。
ガードの上から じわじわと圧をかけて行きますです。
ガードの上から じわじわと五十嵐にプレッシャーをかけていきます。
ガードの上から じわじわと腸だけでいかがですか?
ガードの上から じわっと詰めていきましたです。


tokenizerの中身を確認  
\<s\>の意味合いを表示  


In [ ]:
# model.generateの結果はtokenizerのindexベクトル
output[4]

tensor([    9,  5682,    10, 12276,     2,     9,     0,    20,  2115,    18,
         5456,  2199,     7,    80,     0,    10,   819,     8,     2,     2,
            2,     2,     2])

In [ ]:
# 記号の意味
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [ ]:
# 記号に対応するindex
tokenizer.all_special_ids

[1, 2, 0, 5, 3, 4, 6]

{"bos_token": "\<s\>", "eos_token": "\</s\>", "unk_token": "<unk>", "sep_token": "[SEP]", "pad_token": "[PAD]", "cls_token": "[CLS]", "mask_token": "[MASK]"}  
bos_token: 文の先頭（Begin of sequence token）  
eos_token: 文のおしり（End of Sequence token）  
unk_token: IDに変換できない文字（Unknown token）  
sep_token: 文と文を区切り目（The separator token）  
pad_token: パッディング（The token used for padding）  
cls_token: 分類用（cls_token）  
mask_token: マスク（The token used for masking values）  

* https://huggingface.co/transformers/internal/tokenization_utils.html#transformers.tokenization_utils_base.PreTrainedTokenizerBase.batch_decode  
sequences : torch.Tensorの配列を入力値として指定  
  トークン化された入力IDのリスト  
skip_special_tokens : デコード時に特殊なトークンを削除するかどうか(eos_tokenとかを消す)(デフォルト:False)  
clean_up_tokenization_spaces : トークン化スペースをクリーンアップするかどうか(デフォルト:True)  

* https://huggingface.co/transformers/internal/tokenization_utils.html#transformers.tokenization_utils_base.PreTrainedTokenizerBase.encode  

text (str, List[str] or List[int]) – 入力文字列  

text_pair (str, List[str] or List[int], optional) – ペアとなるもう一つを入力する場合のオプション  

add_special_tokens (bool, optional, defaults to True) – 上記で定義していない特別なトークンをモデルに適用するか

padding (bool, str or PaddingStrategy, optional, defaults to False) –パディングして入力シーケンスを揃える場合  

truncation (bool, str or TruncationStrategy, optional, defaults to False) –逆に長過ぎる場合に、一定の長さに揃える場合

max_length (int, optional) –トランケーション・パディングで使用するオプション

stride (int, optional, defaults to 0) – max_lengthで切り捨てられたのを調整する  

is_split_into_words (bool, optional, defaults to False) – 単語分割が既にされている場合True

pad_to_multiple_of (int, optional) – 指定された値の倍数になるようにシーケンスをパッドする  

return_tensors (str or TensorType, optional) – python整数のリストの代わりにテンソルを返す  
'tf': Return TensorFlow tf.constant objects.  
'pt': Return PyTorch torch.Tensor objects.  
'np': Return Numpy np.ndarray objects.  

* https://huggingface.co/blog/how-to-generate  
* https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.generation_utils.GenerationMixin.generate  
* https://note.com/npaka/n/n5d296d8ae26d  
* https://note.com/npaka/n/n96dde45fdf8d  

### GPT2LMHeadModel.generateのオプションを確認  

input_ids (torch.LongTensor of shape (batch_size, sequence_length), optional) – 入力シーケンス  

max_length (int, optional, defaults to model.config.max_length) – 生成されるシーケンスの最大長を指定（学習に使用した256の長さが良さそう）

max_new_tokens (int, optional, defaults to None) – 現在のトークン数に関係なく、生成されるシーケンスの最大長を指定

min_length (int, optional, defaults to 10) – 生成されるシーケンスの最小の長さ

do_sample (bool, optional, defaults to False) – 単語予測にサンプリングを入れてランダム性を導入する。（デフォルトは greedy decoding の生成）  
https://zenn.dev/hellorusk/articles/1c0bef15057b1d  

early_stopping (bool, optional, defaults to False) – ビーム探索で、num_beams個の文が生成された時点で、ビーム探索を終了するかどうか  

num_beams (int, optional, defaults to 1) – ビームサーチを行うビームの数。1はビームサーチを行わないことを意味します。  

temperature (float, optional, defaults to 1.0) – 次のトークンの確率をモジュール化するために使用される値です。温度（デフォルト1、推奨0.7〜1.0）ボルツマン分布のパラメータ。小さい値ではランダムな補完が減り，0では決まりきった繰り返しの文になる。大きい値ではより様々な補完がされる。  

top_k (int, optional, defaults to 50) – top-k-filteringのために保持する最高確率の語彙トークンの数です。確率が大きめな候補からサンプリングしてランダム性を導入する際の候補を何個にするか。40が一般的に良い値  

top_p (float, optional, defaults to 1.0) – 生成テキストを累積確率に制限 (0で制限なし) float < 1に設定すると、top_p以上の確率を持つ最も確率の高いトークンのみが生成のために保持されます。  
https://zenn.dev/hellorusk/articles/1c0bef15057b1d#top-p-(nucleus)-sampling  

repetition_penalty (float, optional, defaults to 1.0) – 反復ペナルティのパラメータです。1.0はペナルティなし。すでに生成された単語や文脈に属する単語にペナルティを与えるために使用することができます。反復防止にはかなり効果的ですが、異なるモデルやユースケースには非常に敏感なようで、議論がある。  

pad_token_id (int, optional) – PADトークンを指定

bos_token_id (int, optional) – bosトークンを指定

eos_token_id (int, optional) – eosトークンを指定

length_penalty (float, optional, defaults to 1.0) – 長さに対する指数関数的なペナルティ。1.0はペナルティがないことを意味します。1.0未満の値を設定すると、モデルは短い配列を生成するようになり、1.0以上の値を設定すると、モデルは長い配列を生成するようになります。

no_repeat_ngram_size (int, optional, defaults to 0) – int > 0に設定すると、そのサイズのngramはすべて一度しか発生しません。最も一般的な n-grams ペナルティは、すでに見た n-gramsを作る可能性のある次の単語の確率を 0 に手動で設定することで、n-gramsが 2 回出現しないようにするものです。  

encoder_no_repeat_ngram_size (int, optional, defaults to 0) – int > 0に設定すると、encoder_input_idsに出現したそのサイズのすべてのngramは、decoder_input_idsには出現しません。

bad_words_ids (List[List[int]], optional) – 生成してはいけないトークンのidのリスト。トークンのIDは以下で確認  
tokenizer(bad_word, add_prefix_space=True).input_ids  

num_return_sequences (int, optional, defaults to 1) – バッチ内の各要素について、独立して計算された戻り値の配列の数(返却される結果の数)。返されるべき最高得点のBeamの数を設定します。ただし、num_return_sequences <= num_beams とします。

max_time (float, optional, defaults to None) – 計算の実行を許可する最大時間を秒単位で指定します。割り当てられた時間が経過しても、生成は現在のパスを終了します。  

attention_mask (torch.LongTensor of shape (batch_size, sequence_length), optional) – パディングされたトークンのインデックスに対してアテンションを行わないようにするためのマスクです。マスクの値は [0, 1] で、マスクされていないトークンには 1、マスクされたトークンには 0 です。提供されていない場合は、パッドトークンをマスクするinput_idsと同じ形のテンソルがデフォルトになります。attentionで予測するための配列を作るので、マスクすると候補に出なくなる。入力シーケンスに対して同じ長さで指定する

decoder_start_token_id (int, optional) – エンコーダ・デコーダモデルがbosとは異なるトークンでデコードを開始した場合、そのトークンのid。

use_cache – (bool, optional, defaults to True): 過去の最後のキー／バリューの注目度（モデルに該当する場合）を利用して、デコーディングを高速化するかどうか。  

num_beam_groups (int, optional, defaults to 1) – num_beamsを分割するグループの数（ビームの異なるグループ間の多様性を確保するため）。

diversity_penalty (float, optional, defaults to 0.0) – この値は、ある時点で他のグループのビームと同じトークンを生成した場合、ビームのスコアから差し引かれます。なお、ダイバーシティペナルティは、グループビーム検索が有効な場合にのみ有効です。  

prefix_allowed_tokens_fn – (Callable[[int, torch.Tensor], List[int]], optional):提供された場合、この関数は、各ステップで許可されたトークンのみにビーム検索を制約します。提供されない場合、制約は適用されません。この関数は2つの引数をとります：バッチID batch_id と input_id です。これは、バッチID batch_idと以前に生成されたトークンinput_idsを条件として、次の生成ステップで許可されたトークンのリストを返さなければなりません。この引数は、「自己回帰的実体検索」で説明されているように、接頭辞を条件とした制約付き生成に役立ちます。  

output_attentions (bool, optional, defaults to False) – すべてのアテンションレイヤーのアテンションテンソルを返すかどうか。  

output_hidden_states (bool, optional, defaults to False) – すべてのレイヤーの隠れた状態を返すかどうか。  

output_scores (bool, optional, defaults to False) – 予測スコアを返すかどうか。  

return_dict_in_generate (bool, optional, defaults to False) – 単なるタプルではなく、ModelOutputを返すかどうか。  

forced_bos_token_id (int, optional) – decoder_start_token_idの後に、最初に生成されるトークンとして強制的に使用するトークンのidです。mBARTのような多言語モデルで、最初に生成されるトークンがターゲット言語のトークンである必要がある場合に便利です。（一番最初に生成される単語を指定してしまう。）  

forced_eos_token_id (int, optional) – max_lengthに達したときに、最後に生成されたトークンとして強制的に使用するトークンのidです。(最後をわかりやすくして、途中で切られたのを知らせる)  

remove_invalid_values (bool, optional) – 生成方法がクラッシュするのを防ぐために、モデルの可能性のあるnanとinfの出力を削除するかどうか。remove_invalid_valuesを使うと生成が遅くなることに注意してください。  

synced_gpus (bool, optional, defaults to False) – max_lengthまでwhileループを続けて実行するかどうか  

最新の研究により、単純な Beam Search や Greedy Search が同じ単語列の繰り返しを発生させてしまうのは、decoding に問題があるのではなくモデルの学習自体に問題があるとされています。また、Top-K や Top-p のようなサンプリングによる decoding であってもそうした単語列の繰り返しは発生しうるそうです。  

In [9]:
input = tokenizer.encode("左のフック", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のフック</s> ボディに打ち込んでいく<unk> 、そして、ショートレンジのパンチ。<unk> が入ってきてもパンチで応戦して、逆に押し返しているのが<unk> ですかね。</s> </s> </s> </s> </s> </s> </s> </s> </s>
左のフック</s> 少しふらついた!<unk>!少し効いているか!効いている!右のストレート!さあ今度は<unk> の逆襲!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のフック</s> もうちょっと<unk> 、スタミナ厳しいですから気持ちでいいので お休みしますがまだ入るとのこと。 体力ものこっていますし。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のフック</s> 試合序盤からハグが積極的に前に行きますこのサウスポーの選手赤のグローブ昨年の2月にデビューしそこで判定負けを喫してしまった泡とみなされた。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のフック</s> 左のボディをいれてきた<unk>!<unk>!ロープ際!一気に攻勢を強めていきます<unk> です。 さあ、ここまでは優勢に試合を進めている<unk> ですが、</s> </s> </s> </s> </s> </s> </s>
左のフック</s> パロディ30秒に一番近いを返さない。 バビロン。 ひらりひらり入った幼い。 残り10秒を切りました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のフック</s> <unk> 選手今の顔もそうなんですが、落ち着いていますね。さあ行くぞみたいな顔してたんで。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のフック</s> 赤コーナーよりサカタケントがこれから入場をして来ようとしています。

In [10]:
input = tokenizer.encode("ラッシュラッシュ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ラッシュラッシュ</s> <unk> 選手今の顔もそうなんですが、落ち着いていますね。さあ行くぞみたいな顔してたんで。さあ行くぞみたいな顔してたんで。</s> </s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> <unk> 選手はそれを狙っている様子もありますかね。 今のところは狙ってない感じですかね。そこへボディを打ち込んでいく<unk> 。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> <unk> 選手はそれを狙っている様子もありますかね。 サウスポーにとってちょっと嫌な打ち下ろしの左ストレートでくるんですけど、そこが長所であり、あえて狙うならばそこが短所になるので、</s>
ラッシュラッシュ</s> <unk> 選手はそれを狙っている様子もありますかね。 今のところは狙ってない感じですかね。そこへボディを打っていくのは<unk> ですか。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> <unk> 選手今の顔もそうなんですが、落ち着いていますね。さあ行くぞみたいな顔してたんで。さあ行くぞみたいな顔してたんで。</s> </s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> <unk> 選手はそれを狙っている様子もありますかね。 今のところは狙ってない感じですかね。そこへボディを打っていくのは<unk> ですか。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> <unk> 選手はそれを狙っている様子もありますかね。 サウスポーにとってちょっと嫌な打ち下ろしの左ストレートでくるんですけど、そこが長所であり、あえて狙うならばそこが短所になるので、</s>
ラッシュラッシュ</s> <unk> 選手今の顔もそうなんですが、落ち着いていますね。さあ行くぞみたいな顔してたんで。さあ行くぞみたいな顔

In [11]:
input = tokenizer.encode("ダウン", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ダウン</s> <unk> 。 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。</s> </s>
ダウン</s> <unk> 。 どんどん前に出てきがパンチが出ないでみんなそれぞれの山頂開始しました教えてきたがすごい簡単ものラウンドなかなかパンチが出せなかった。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ダウン</s> <unk> 。 第5ラウンドが終了しました。 いかがでしたか今のラウンド。お互い<unk> 選手はワンツーがめっちゃヒットし、<unk> 選手はこのボディとフック、左右、いろんな角度のついた感じで、ではい確認してましたね</s>
ダウン</s> <unk> 。 第6ラウンドが終了しました。 いかがでしたか今のラウンド。お互い<unk> 選手はワンツーがめっちゃヒットし、<unk> 選手はこのボディとフック、左右、いろんな角度のついた感じで、ではい確認してましたね</s>
ダウン</s> <unk> ました。<unk> 。 どんどん前に出てきがパンチが出ないでみんなそれぞれの山頂開始しました教えてきたがすごい簡単ものラウンドなかなかパンチが出せなかった。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ダウン</s> <unk> 。 さあ、この辺りの入りですけれども、<unk> Cもまずは丁寧にボクシング文だっていきたいですし、<unk> 選手も丁寧にボクシング文だっていきたいですね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ダウン</s> <unk> 。 どんどん前に出てきがパンチが出ないでみんなそれぞれの山頂開始しました教えてきたがすごい簡単もの

In [12]:
input = tokenizer.encode("あーとここで倒れた", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


あーとここで倒れた</s> 少しふらついた!左!少しふらついた!最後は左のボディ!やはりパンチには威力があります!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> 少しふらついた!左!少しふらついた!最後は左のボディ!やはり両者の戦い!距離の測り愛といったところでしょうか。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </

In [13]:
input = tokenizer.encode("ここで倒れた", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.85, temperature=0.7, min_length=64, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ここで倒れた</s> 少しふらついた! やはり前に出ますね。 ただ引く感じではないと思うので、最終ラウンド、打ち合い覚悟で前に出てくるでしょう。 ジャブを打って、最後のスタミナを振り絞ってという最終ラウンドになりますかね。 やっぱり最終ラウンドになってもしっかりしているのでよ</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ここで倒れた</s> 先ほどの神ですこの右のストレート中に入り込もうというところです。 今リング中央には神がいます。 今リングインです。 浜田麻里奈エスペランサジム所属です日本女子フライ級8位の選手今回は階級を下げての下げての今回試合となりました減量期間は少し長かったんですが試合のために減量頑張れたと甘いものが好きなんだとただそれを我慢するのが辛いんですがダイエットになるから意外と楽しめたと語っていましたがまだです。</s>
ここで倒れた</s> 倒れなかった選手はそれに対して何が大事になってきますか川畑さんランドのあのやりにくいボクシングハイができたらいいかなと思いますね特に今流れを一度持ってかれそうになった後に切り替えるそのクレバーさもありましたよしっかり相手を見て試合をしてたと思いますこれが3杯目ということになりましたてらさきかずきただ頭を使うボクシングその磨いてきた練習の一環というのはこの試合も示せたようにも思いますがいかがでしょうか僕はすごくそう思いますね。</s> </s> </s> </s> </s> </s> </s>
ここで倒れた</s> 倒れなかった選手も大概痛いですよね。立てないですよね。立てないですね。ただ、ただ倒れなかった選手も大概痛いですよね。立てないですね。ただ、倒れなかった選手もだいぶ効いていると思うので、このまま行けたらいいと思います。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s

In [14]:
input = tokenizer.encode("ガードの上から連打", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ガードの上から連打</s> 早くもコンビネーション!<unk> も返していきます!両者相打つ!右!右のジャブ!<unk> も返していきます!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上から連打</s> 早くも浪速のターミネーター48なるか連れて行ってますよね今も千鳥届いたでしょうかになってますね藤井はどうするかで画像これから出していこうというのがそのうち終わりをやらは狙っていますそれでも藤井はジャムなども含めて出して行こうとしています左利きにかけてパンツを出していてコンビネーション1割を狙っているそんなシーンもありましたこの砂利はこの集団は誰が出しましたそのパンチもご覧いただきたいと思いますが。</s>
ガードの上から連打</s> 早くも中村!左のジャブ!これも左! 今成も左のジャブを繰り出していきます。 第4ラウンドももう残り1分を切っています。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s

In [15]:
input = tokenizer.encode("倒れたノックアウトです。", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


倒れたノックアウトです。</s> 両者ともに意地と意地のぶつかり合いパンチの打ち合いになっている気持ちの勝負だ気持ちの勝負念願のプロのリングお互いに意地と意地のぶつかり合いファミリーのプレート試合を止めました。</s> </s> </s>
倒れたノックアウトです。</s> 両者ともに意地と意地のぶつかり合いパンチの打ち合いになっている気持ちの勝負だ気持ちの勝負念願のプロのリングお互いに意地と意地のぶつかり合いファミリーのプレート試合を止めました。</s> </s> </s>
倒れたノックアウトです。</s> 両者ともに意地と意地のぶつかり合いパンチの打ち合いになっている気持ちの勝負だ気持ちの勝負念願のプロのリングお互いに意地と意地のぶつかり合いファミリーのプレート試合を止めました。</s> </s> </s>
倒れたノックアウトです。</s> 倒れたまま第4ラウンドが終了しました。 左のフック!ボディに打ち込んでいく<unk>!ロープ際! <unk> が勝利を収めました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れたノックアウトです。</s> 両者ともに意地と意地のぶつかり合いパンチの打ち合いになっている気持ちの勝負だ気持ちの勝負念願のプロのリングお互いに意地と意地のぶつかり合いファミリーのプレート試合を止めました。</s> </s> </s>
倒れたノックアウトです。</s> 両者ともに意地と意地のぶつかり合いパンチの打ち合いになっている気持ちの勝負だ気持ちの勝負念願のプロのリングお互いに意地と意地のぶつかり合いファミリーのプレート試合を止めました。</s> </s> </s>
倒れたノックアウトです。</s> 両者ともに意地と意地のぶつかり合いパンチの打ち合いになっている気持ちの勝負だ気持ちの勝負念願のプロのリングお互いに意地と意地のぶつかり合いファミリーのプレート試合を止めました。</s> </s> </s>
倒れたノックアウトです。</s> 倒れたまま大丈夫でしょうかこれから担架で運ばれるかというところです。 今リングサイドで様子を見ています高橋義生第4ラウンドが終了しました結構盛り上がったと思いますこのまま展開を続けていくんでし

In [16]:
input = tokenizer.encode("倒れたKOです。", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


倒れた<unk> です。</s> 先ほどの神ですこの右のストレート中に入り込もうというところです。 やはりハードパンチャーですから一発大きいのがあればかなりの威力がありますこの赤コーナーの満田です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れた<unk> です。</s> 先ほどの神ですこの右のストレート中に入り込もうというところです。 やはりハードパンチャーですから一発大きいのがあればかなりの威力がありますこの赤コーナーの満田です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </

In [17]:
input = tokenizer.encode("右のフックから左右連打", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


右のフックから左右連打</s> 少しボディが効いているか動きが遅くなってそれでも右のフックを打っています。 スイッチしましたね住吉右のフックボディに受けましたもう一度直して。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のフックから左右連打</s> 少しボディが効いているか動きが遅くなってそれでも右のフックを打っています。 スイッチしましたね住吉右のフックボディに受けましたもう一度直して。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のフックから左右連打</s> コンビネーションも見えてきました。<unk> もこの距離は許さないというな構えを見せます。 第3ラウンドもかなり少なくなってきています。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </

In [18]:
input = tokenizer.encode("ガードの上からボディー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ガードの上からボディー</s> 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。</s>
ガードの上からボディー</s> 左のジャブで<unk> Cは真正ジムに移籍してこれが初戦ということになります。 <unk> 会長に何としてもお礼を言いたい、そういった思いで戦うこの一戦です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上からボディー</s> 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。</s>
ガードの上からボディー</s> 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。</s>
ガードの上からボディー</s> 左で距離を取りますヒヤリ。 第3ラウンドももう残り1分を切っています右この独特な右がありますよな晴れ。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上からボディー</s> 左で距離を取りますヒヤリ。 第3ラウンドももう残り1分を切っています右この独特な右がありますよな晴れ。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上からボディー</s> 左のジャブで<unk> Cは真正ジムに移籍してこれが初戦ということになります。 <unk> 会長に何としてもお礼を言いたい、そういった思いで戦うこの一戦です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>

In [19]:
input = tokenizer.encode("ジャブ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ジャブ</s> ストレート <unk> 左の方にとって行きました。 次のストレートを打つ満田の画面取れたか少し聞いたか。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ジャブ</s> ストレート <unk> とどうでしょうかて画像出していますが頭のてっぺん高いですよねあれがやはりハードパンチャーですから一発大きいのがあればかなりの威力がありますこの赤コーナーの満田です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ジャブ</s> ストレート <unk> みき みき <unk> この辺りの戦略というのはどうでしょうか? 今のところはそうですね、<unk> 選手が自分のペースに持って行こうとしたら、<unk> は逆に押し返してしまって、逆に押し返されてこそ、<unk> 選手のチャンピオンとしての真価が問われると思うんです。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ジャブ</s> ストレート <unk> を打って行きます。 少し距離を詰めていったこの両者、第3ラウンド流れが変わるかもしれません。</

In [20]:
input = tokenizer.encode("左のジャブ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のジャブ</s> サウスポーにとってちょっと嫌な打ち下ろしの左ですね。 そこが長所であり、あえて狙うならばそこが短所になるので、</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブ</s> ピリピリとしたこの空気感とコンプレックスです本当にお互いがボクシング人生にかけているって言うならない選手は試合の序盤にダウンを奪われましたが第5ラウンドあたりからね自分の距離で戦えるようになる自分だけじゃないかもしれないですけどもあいつの強さで自分のペースに持って行こうとしているのがちょっと見えるかなと僕は思いまなるほど本来はもう着く距離を取り込んだと思う。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブ</s> プレッシャーをかけているのは山中です。ボディに打ち込んでいく。持ち味は左のフックと話しているこの山中ですが、まだ教育写真のここまでは出来ませんありませんね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </

In [21]:
input = tokenizer.encode("右のジャブ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


右のジャブ</s> いきなり少し変化がありましたか。まだ立てませんは来た。 いきなり第2ラウンド終了です。非常に激しい打ち合いになっている。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のジャブ</s> いきなり少し変化がありましたでしょうか第2ラウンド終了です。 今のところは谷山がダウンを奪いましたから攻め続けた谷山が第2ラウンドでダウンを奪っています。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のジャブ</s> いきなり少し変則的な動きになりましたが。。 <unk> ですかね。 今のところはそうだと思います。 ただ、<unk> の持ち味は左のフックと話しているので、そこがコンビネーションの一つのポイントになっています。</s>
右のジャブ</s> いきなり少し変則的なステップになりましたが。。 <unk> ですかね。 今のところはそうだと思います。 今のところは<unk> がプレスをかけていこうというところですが右のジャブを打っています。</s> </s> </s>
右のジャブ</s> いきなり少し変則的な動きになりましたが。。 <unk> ですかね。 今のところはそうだと思います。 今のところは<unk> がプレスをかけていこうというところですかね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のジャブ</s> チリチリとプレッシャーをかけるきらりのストレートを返していきます。 プレッシャーをかけているのは渡辺の方ですかね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のジャブ</s> 左のジャブを繰り出してきたのは長いですが持ち味は左のフックと話しているこの長いですがまだ教育写真のここまでは出来ませんありませんねまだ出てきたこのぐらいの距離です。</s> </s> </s> </s> </s> </s> </s>
右のジ

In [22]:
input = tokenizer.encode("左のジャブからワンツー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のジャブからワンツー</s> コンビネーションを見せました。 やはり先手を取っているのは<unk> ですかね。 今のところはそうだと思います。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブからワンツー</s> やはりパンチには威力があります。<unk> 選手はそれを狙っている様子もありますかね。 サウスポーにとってちょっと嫌な打ち下ろしの左ストレートでくるんですけど、そこが長所であり、あえて狙うならばそこが短所になるので、</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブからワンツー</s> 試合序盤からハグが積極的に前に行きますこのサウスポーの選手赤のグローブ昨年の2月にデビューしそこで判定負けを喫してしまった泡とみなされた。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>

In [23]:
input = tokenizer.encode("左のアッパー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のアッパー</s> <unk> 選手はそれを狙っている様子もありますかね。 第4ラウンド終了です。 いきなり左。うまいですね。思い切ってうってきたところを、どうですか今のところは<unk> も返して行きます。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のアッパー</s> ジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のアッパー</s> トロフィーはここ早く外したいところですよね。 ここも体を当てて行きます残り30秒た世界タイトル初防衛世界タイトル初挑戦軍配が上がるのはどちらか気持ちを見せ続けてきた時そのチャレンジャーお迎え行ってきた今から10月が前に出る月が前に出る。</s>
左のアッパー</s> リーペペレースボディに打ち込んでいく<unk> 、そして、ショートレンジのパンチ。<unk> が入ってきてもパンチで応戦して、逆に押し返しているのが<unk> ですかね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のアッパー</s> ボディに打ち込んでいく<unk> 、そして、ショートレンジのパンチ。<unk> が入ってきてもパンチで応戦して、逆に押し返しているのが<unk> ですかね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のアッパー</s> ボディに打ち込んでいく<unk> 、そして、ショートレンジのパンチ。<unk> が入ってきてもパンチで応戦して、逆に押し返しているのが<unk> ですかね。</s> </s> 

In [24]:
input = tokenizer.encode("右のアッパー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


右のアッパー</s> 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> <unk> 選手はそれを狙っている様子もありますかね。 今のところはそうだと思います。 第3ラウンド終わりました。 ちょっと様子見という第1ラウンドから第二ラウンド、様子見ながら拳を交えながら、というところがありますね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> 

In [25]:
input = tokenizer.encode("アッパー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


アッパー</s> 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。</s>
アッパー</s> 左ボディをやってきました長いです先ほどからリズムが予算良くなってきたという話もありました。 んじゃないかなとは思いますねこの辺りのこの上下の打ち分けその入る時のスタンドさんのストレートハイ。</s> </s> </s> </s> </s> </s>
アッパー</s> 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。</s>
アッパー</s> 左ボディをやってきました<unk> です。 <unk> がどう攻めていくか。 第7ラウンドまで終了しています。 二人の一戦は最終ラウンドへと移っていきます。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
アッパー</s> 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。</s>
アッパー</s> 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。</s>
アッパー</s> 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。</s>
アッパー</s> 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディお花って言ったときょうへい中川も左のジャブを繰り出していきます。</s>
アッパー</s> 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような

In [26]:
input = tokenizer.encode("カウンター", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


カウンター</s> <unk> 選手はそれを狙っている様子もありますかね。 サウスポーにとってちょっと嫌な打ち下ろしの左ストレートでくるんですけど、そこが長所であり、あえて狙うならばそこが短所になるので、</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
カウンター</s> みかとはよく寝つきを良くしていたんで吐いたりやりにくい距離感がピンボケするのではいよく言われる距離感が難しいというところなんですけども野上さんその辺りはいかがですかそうですねやはり常にオーソドックスと戦うことが多いのでそれを散歩に切り替えられたりっていうことはとてもやりにくい状況だと思いますもちろん鈴木奈々イモトのスイッチとは今回が初対戦になります第一ランドまもなくゴングというところ。</s>
カウンター</s> <unk> 選手今の顔もそうなんですが、落ち着いていますね。さあ行くぞみたいな顔してたんで。さあ行くぞみたいな顔してたんで。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
カウンター</s> <unk> 選手はそれを狙っている様子もありますかね。 サウスポーにとってちょっと嫌な打ち下ろ

In [27]:
input = tokenizer.encode("試合展開", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


試合展開</s> ピリピリとしたこの空気感とコンプレックスです本当にお互いがボクシング人生にかけているって言うならない選手は試合の序盤にダウンを奪われましたが第5ラウンドあたりからね自分の距離で戦えるようになる自分だけじゃないかもしれないですけどもあいつの強さで自分のペースに持って行こうとしているのがちょっと見えるかなと僕は思いまなるほど本来はもう着く距離を取り込んだと思う。</s>
試合展開</s> ピリピリとしたこの空気感とコンプレックスです本当にお互いがボクシング人生にかけているって言うならない選手は試合の序盤にダウンを奪われましたが第5ラウンドあたりからね自分の距離で戦えるようになる自分だけじゃないかもしれないですけどもあいつの強さで自分のペースに持って行こうとしているのがちょっと見えるかなと僕は思いまなるほど本来はもう着く距離を取り込んだと思う。</s>
試合展開</s> ピリピリとしたこの空気感とコンプレックスです本当にお互いがボクシング人生にかけているって言うならない選手は試合の序盤にダウンを奪われましたが第5ラウンドあたりからね自分の距離で戦えるようになる自分だけじゃないかもしれないですけどもあいつの強さで自分のペースに持って行こうとしているのがちょっと見えるかなと僕は思いまなるほど本来はもう着く距離を取り込んだと思う。</s>
試合展開</s> いつも通りボクシングをして、いつも通り勝利をつかみたいと伺っていました。 <unk> です。 いつも通り練習通りのボクシングをし、いつも通りの勝利を掴みたいと伺っていました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開</s> ピリピリとしたこの空気感とコンプレックスです本当にお互いがボクシング人生にかけているって言うならない選手は試合の序盤にダウンを奪われましたが第5ラウンドあたりからね自分の距離で戦えるようになる自分だけじゃないかもしれないですけ

In [28]:
input = tokenizer.encode("試合展開について", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


試合展開について</s> 少し<unk> 腸を出してきたのでたですはじめの一歩の左上あたりの部分が赤くなっていますでしょうか距離は少し詰まっていますでしょうかはじめの一歩の左上あたりの部分が赤くなっています。</s>
試合展開について</s> 少し<unk> 、スタミナ厳しいですから気持ちでいいのでどんどん前に出ていくたいですね。 そこにボディを打ち込んでいく<unk> 。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開について</s> 少し<unk> 、スタミナ厳しいですから気持ちでいいのでどんどん前に出ていくスタミナ勝負に持ち込みたいと語っていました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開について</s> 少し<unk> 、スタミナ厳しいですから気持ちでいいので お休みしますがまだ入るとのこと。 体力ものこっていますし。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開について</s> 少し<unk> 、スタミナ厳しいですから気持ちでいいのでほしいですねそうですね私も残り10秒、最後力を振り絞ってほしいですね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開について</s> 少し<unk> 、スタミナ厳しいですから気持ちでいいので お休みしますがまだ入るとのこと。 体力ものこっていますし。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開について</s> 少し<unk> 、スタミナ厳しいですから気持ちでいいので お休みしますがまだ入るとのこと。 体力ものこっていますし。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開について</s> 少し<unk>

In [29]:
input = tokenizer.encode("ボクシング", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ボクシング</s> 左のジャブが伸びている。 第3ラウンドももう残り1分を切っています右のストレート。 さあ山中さんあい最後は今それと出しましたねまだ石巻市に来てますね。</s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> 56キロ契約で行われます8回戦。 まずは飛び出したのが<unk> です。 やはり元アマチュアの選手なので距離を取って隙のない選手と思っていたのですが<unk> ですが、そういった展開、</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> 56キロ契約で行われます8回戦。 まずは飛び出したのが<unk> です。 やはり元アマチュアの選手なので距離を取って隙のない選手と思っていたのですが<unk> ですが、そういった展開、</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> 56キロ契約で行われます8回戦。 まずは飛び出したのが<unk> です。 やはり元アマチュアの選手なので距離を取って隙のない選手と思っていたのですが<unk> ですが、そういった展開、</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> 56キロ契約で行われます8回戦。 まずは飛び出したのが<unk> です。 これまで3戦は2勝1分け、対する<unk> は6戦2勝。2勝のうち一つは<unk> 勝利、3連敗からの2連勝もありました。</s>
ボクシング</s> 56キロ契約で行われます8回戦。 <unk> 対<unk> の一戦が始まってすぐにゴングが鳴り響きました。 赤いボクシンググローブが<unk> です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> 左のジャブを打って行きました青いグローブ能登元恭平ラスそして中川は今両手を広げて胸を見せるような仕草を見せました左のボディ

In [30]:
input = tokenizer.encode("ボクシングの試合について", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ボクシングの試合について</s> <unk> 選手今の顔もそうなんですが、落ち着いていますね。さあ行くぞみたいな顔してたんで。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> <unk> 選手はジムでも愛されキャラでお出かけスタイル的に気持ちが強い選手なので相手の選手もききっちに来て。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> <unk> 選手今の顔もそうなんですが、落ち着いていますね。さあ行くぞみたいな顔してたんで。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> <unk> 選手今の顔もそうなんですが、落ち着いていますね。さあ行くぞみたいな顔してたんで。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> <unk> 選手今の顔もそうなんですが、落ち着いていますね。さあ行くぞみたいな顔してたんで。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> <unk> 選手今の顔もそうなんですが、落ち着いていますね。さあ行くぞみたいな顔してたんで。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について<

In [31]:
input = tokenizer.encode("今回のボクシングの試合について", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


今回のボクシングの試合について</s> <unk> 選手は前回の試合がキャリア初の黒星そして初の<unk> 勝利でしたそこからの再起戦ということになります今日の試合も前回と同じ気持ちで試合に臨んでくるでしょうか<unk> 選手。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
今回のボクシングの試合について</s> チャンピオンの岩川もかなり肩で息していますからたださんこれやはり岩川にとってもなかなか厳しいラウンドになってますよねそうですねあの体のサイズが違う中でチャンピオンやっぱりもうちょっとショートレンジャーあの出していくっていうなんか工作がないとパーク in は覚えてきたので慌てていくっていう必要性はありませんあの鈴木選手のあの本当にあのボディはい中でその中のストーリーの一つのパンチがないのではいちょっと鈴木選手が戦い方に今ちょっと揉まれて言ってるのになるほどですから鈴木が体を密着させる前の所で何か一つこの距離感という間でしたねこの避けていく中で何かチャンピオンは一つビックパンチが欲しいところ。</s>
今回のボクシングの試合について</s> <unk> は本当に落ち着いて試合展開、ただ強気のボクシングを見せている。 中盤にかけてその<unk> に対して、キ

In [32]:
input = tokenizer.encode("かなりのダメージを受けている", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


かなりのダメージを受けている</s> <unk> 選手はそれを狙っている様子もありますかね。 サウスポーにとってちょっと嫌な打ち下ろしの左ストレートでくるんですけど、そこが長所であり、あえて狙うならばそこが短所になるので、</s>
かなりのダメージを受けている</s> さあ。 今出ている距離を詰めてパンチを出せるかの時は新治ボディにパンチを集めている開き状態をやって距離を詰めて高橋をしても左を出していく。</s> </s> </s> </s> </s> </s> </s> </s> </s>
かなりのダメージを受けている</s> そこへボディを打つ!<unk>! やはりパンチには威力があります。<unk> です。 今成もパンチを返してきたらどんどん前に出ていく、ボディです。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
かなりのダメージを受けている</s> そこまで考えて両者ともに飛び出したこの両者の戦い。 パンチの打ち合いになりました。 右のフックから右のストレート、これが一つ鍵になってきます。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
かなりのダメージを受けている</s> そこまで考えてコンビネーションを打つと。。 少しパンチが大きくなっているが、返して行きます<unk> です。ワンツーと打っていく。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
かなりのダメージを受けている</s> 先ほどの神ですこの右のストレート中に入り込もうというところです。 今リングギアに追い込まれていますが何とかならなかったですね危なかったところでしたいやここは両者。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
かなりのダメージを受けている</s> <unk> です。 さあ、まず序盤から前に出ようというのは<unk> ですかね。 そうですね、もうちょっと<unk> 自身も予想外なことをされたのかなっていう感じにはなりますけども。</s> </s> </s> </s>
かなりのダメージを受けている<